In [ ]:
!pip install deepface tf-keras matplotlib
from deepface import DeepFace
from retinaface import RetinaFace

In [15]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [14]:
face_objs = DeepFace.extract_faces(
  img_path = "test.jpg",
  detector_backend = "dlib",
  align = True,
)
face_objs

25-02-11 09:13:32 - shape_predictor_5_face_landmarks.dat.bz2 is going to be downloaded


Downloading...
From: http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2
To: /root/.deepface/weights/shape_predictor_5_face_landmarks.dat.bz2
100%|██████████| 5.71M/5.71M [00:00<00:00, 18.6MB/s]


[{'face': array([[[0.08235294, 0.06666667, 0.07058824],
          [0.09019608, 0.0745098 , 0.07843137],
          [0.09019608, 0.0745098 , 0.07843137],
          ...,
          [0.05882353, 0.03921569, 0.02745098],
          [0.0627451 , 0.04313725, 0.03137255],
          [0.05882353, 0.03921569, 0.02745098]],
  
         [[0.07843137, 0.0627451 , 0.06666667],
          [0.09019608, 0.0745098 , 0.07843137],
          [0.08627451, 0.07058824, 0.0745098 ],
          ...,
          [0.0627451 , 0.04313725, 0.03137255],
          [0.05882353, 0.03921569, 0.02745098],
          [0.05882353, 0.03921569, 0.02745098]],
  
         [[0.07843137, 0.0627451 , 0.06666667],
          [0.08235294, 0.06666667, 0.07058824],
          [0.08627451, 0.07058824, 0.0745098 ],
          ...,
          [0.0627451 , 0.04313725, 0.03137255],
          [0.05882353, 0.03921569, 0.02745098],
          [0.05490196, 0.03529412, 0.02352941]],
  
         ...,
  
         [[0.8       , 0.53333333, 0.35686275],
      

In [ ]:
img = cv2.imread('test.jpg')
plt.imshow(img[:, :, ::-1])

In [18]:
type(face_objs)

list